# CISA KVE EPSS Data Analyis 

In [1]:
import pandas as pd
import json
import requests
import os
import glob
import numpy as np

In [2]:
cisa_df = pd.read_csv("https://www.cisa.gov/sites/default/files/csv/known_exploited_vulnerabilities.csv")
cisa_df = cisa_df
cisa_df.columns = cisa_df.columns.str.strip("\u200b")
cisa_df = cisa_df.rename(columns={"cveID": "CVE", "shortDescription" : "Description", "dateAdded" : "Date"})

In [3]:
epss = pd.read_csv('epss_scores-current.csv', skiprows=1)
epss = epss.rename(columns={"cve": "CVE", "epss" : "EPSS", "percentile" : "EPSS Percentile"})

In [4]:
row_accumulator = []
for filename in glob.glob('nvdcve-2.0-*.json'):
    with open(filename, 'r', encoding='utf-8') as f:
        nvd_data = json.load(f)
        for entry in nvd_data['vulnerabilities']:
            cve = entry['cve']['id']
            try:
                base_score = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['baseScore']
            except KeyError:
                base_score = '0.0'
            new_row = { 
                'CVE': cve, 
                'CVSS3': base_score,
            }
            row_accumulator.append(new_row)
        nvd = pd.DataFrame(row_accumulator)
        
nvd['CVSS3'] = pd.to_numeric(nvd['CVSS3']);
nvd['CVSS3'] = nvd['CVSS3'].replace(0, np.nan);       

In [5]:
epss_kev = pd.merge(cisa_df, epss, left_on='CVE', right_on='CVE')
epss_kev_nvd =  pd.merge(epss_kev, nvd, left_on='CVE', right_on='CVE')
epss_kev_nvd =  epss_kev_nvd[["CVE", "CVSS3", "EPSS", "EPSS Percentile", "Description", "Date"]]

## CISA KEV Score Scatter Plot

/opt/hostedtoolcache/Python/3.10.18/x64/lib/python3.10/site-packages/pandas/plotting/_matplotlib/core.py:1351: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  scatter = ax.scatter(


In [6]:
ax = epss_kev_nvd.plot.scatter(x='CVSS3',
                     y='EPSS',
                     colormap='jet',
                     figsize=(20, 10),
                     title='CISA Known Exploited Vulnerabilities');
ax.set_xlabel("CVSS 3 Base Score");
ax.set_ylabel("EPSS Score");
ax.figure.savefig('epss_kev_nvd.png');

## Export to CSV

In [7]:
epss_kev_nvd.to_csv("epss_kev_nvd.csv", index=False)
epss_kev_nvd.head(10)

,CVE,CVSS3,EPSS,EPSS Percentile,Description,Date
0,CVE-2025-32463,9.3,0.24359,0.95930,Sudo contains an inclusion of functionality fr...,2025-09-29
1,CVE-2025-59689,6.1,0.21269,0.95516,Libraesva Email Security Gateway (ESG) contain...,2025-09-29
2,CVE-2025-10035,10.0,0.81064,0.99125,Fortra GoAnywhere MFT contains a deserializati...,2025-09-29
3,CVE-2025-20352,7.7,0.00893,0.74876,Cisco IOS and IOS XE contains a stack-based bu...,2025-09-29
4,CVE-2021-21311,7.2,0.94150,0.99912,Adminer contains a server-side request forgery...,2025-09-29
5,CVE-2025-20362,6.5,0.00812,0.73571,Cisco Secure Firewall Adaptive Security (ASA) ...,2025-09-25
6,CVE-2025-20333,9.9,0.01034,0.76700,Cisco Secure Firewall Adaptive Security (ASA) ...,2025-09-25
7,CVE-2025-10585,9.8,0.04962,0.89253,Google Chromium contains a type confusion vuln...,2025-09-23
8,CVE-2025-5086,9.0,0.64220,0.98390,Dassault Systèmes DELMIA Apriso contains a des...,2025-09-11
9,CVE-2025-38352,7.4,0.00580,0.68083,Linux kernel contains a time-of-check time-of-...,2025-09-04
